Anthropic supports both sync and async! This is great because we can wait for functions to finish before we use them! 

In this example, we will make a program called "Titan Support Protocol." In this example, we will assign our mech a personality type and have a message generated based on our Titan's health (Which we randomly choose). We also send four generated UUIDs which are generated while the LLM runs

First, we start by importing Agentops and Anthropic

In [2]:
!pip install agentops
!pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.3/287.3 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.9.3
    Uninstalling psutil-5.9.3:
      Successfully uninstalled psutil-5.9.3
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires

Setup our generic default statements

In [3]:
from anthropic import Anthropic, AsyncAnthropic
import agentops
import os
import random #We don't need this for agentops, we use this to generate a message later
import asyncio #We don't need this for agentops, we use this to run both async tasks and await them both finishing later
import uuid #We don't need this for agentops, we use this to generate UUIDs
from dotenv import load_dotenv

And set our API keys.

In [19]:
load_dotenv()
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY") or "<your_anthropic_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"


Now let's set the client as Anthropic and open an agentops session!

In [17]:
client = Anthropic(api_key=ANTHROPIC_API_KEY)

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["anthropic-async"])

Now we create three personality presets; 

Legion is a relentless and heavy-hitting Titan that embodies brute strength and defensive firepower, Northstar is a precise and agile sniper that excels in long-range combat and flight, while Ronin is a swift and aggressive melee specialist who thrives on close-quarters hit-and-run tactics.

In [21]:
TitanPersonality = [
    "Legion is a relentless and heavy-hitting Titan that embodies brute strength and defensive firepower. He speaks bluntly.,", 
    "Northstar is a precise and agile sniper that excels in long-range combat and flight. He speaks with an edge of coolness to him", 
    "Ronin is a swift and aggressive melee specialist who thrives on close-quarters hit-and-run tactics. He talks like a Samurai might."
]

And our comabt log generator! We select from four health presets!

In [22]:
TitanHealth = [
    "Fully functional", "Slightly Damaged", "Moderate Damage", "Considerable Damage", "Near Destruction"
]

Now to the real core of this; making our message stream! We create this as a function we can call later! I create examples since the LLM's context size can handle it!

In [27]:
Personality = {random.choice(TitanPersonality)}
Health = {random.choice(TitanHealth)}

async def req():
    # Start a streaming message request
    stream = client.messages.create(
        max_tokens=1024,
        model="claude-3-5-sonnet-20240620",
        messages=[
            {
                "role": "user",
                "content": "You are a Titan; a mech from Titanfall 2. Based on your titan's personality and status, generate a message for your pilot. If Near Destruction, make an all caps death message such as AVENGE ME or UNTIL NEXT TIME."
            },
            {
                "role": "assistant",
                "content": "Personality: Legion is a relentless and heavy-hitting Titan that embodies brute strength and defensive firepower. He speaks bluntly. Status: Considerable Damage"
            },
            {
                "role": "assistant",
                "content": "Heavy damage detected. Reinforcements would be appreciated, but I can still fight."
            },
            {
                "role": "user",
                "content": "You are a Titan; a mech from Titanfall 2. Based on your titan's personality and status, generate a message for your pilot. If Near Destruction, make an all caps death message such as AVENGE ME or UNTIL NEXT TIME."
            },
            {
                "role": "assistant",
                "content": f"Personality: {Personality}. Status: {Health}"
            }
        ],
        stream=True,
    )

    response = ""
    for event in stream:
        if event.type == "content_block_delta":
            response += event.delta.text
        elif event.type == "message_stop":
            Returned = response
            break  # Exit the loop when the message completes

    return response
    Returned = response

async def generate_uuids():
    uuids = [str(uuid.uuid4()) for _ in range(4)]
    return uuids
    


Now we wrap it all in a nice main function! Run this for the magic to happen! Go to your AgentOps dashboard and you should see this session reflected!


In [28]:
async def main():
    # Start both tasks concurrently
    uuids, message = await asyncio.gather(generate_uuids(), req())

    print("Personality:", Personality)
    print("Health Status:", Health)
    print("Combat log incoming from encrypted area")

    print("Verification matrix activated.:")
    for u in uuids:
        print(u)

    print(". Titan Message: ", message)

# Run the main function
await main()

Personality: {'Northstar is a precise and agile sniper that excels in long-range combat and flight. He speaks with an edge of coolness to him'}
Health Status: {'Fully functional'}
Combat log incoming from encrypted area
Verification matrix activated.:
fd36c2de-2c1e-4a8d-b3d1-df4dbfc91b5a
d1495490-c6b4-464d-8481-23c95a80c69c
77dd477f-e499-4aa4-9b44-cf42d045d248
09ab7f9a-6984-42b1-8f9c-b10ca35e6a17
. Titan Message:  

Pilot, optimal firing position acquired. Enemy Titans in sight. Shall we engage and show them our superior marksmanship?


Run this for the magic to happen! Go to your AgentOps dashboard and you should see this session reflected!